In [4]:
# =======================================================
# 1) Setup and imports
# =======================================================
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from scipy.signal import butter, lfilter

torch.manual_seed(42)
np.random.seed(42)

# =======================================================
# 2) Config
# =======================================================
BASE_PATH = '/kaggle/input/mtcaic3'
CHANNELS = ['FZ', 'C3', 'CZ', 'C4']
FS = 250
MU_BAND = (8, 13)
BETA_BAND = (13, 30)
MAX_LEN = 2250  # pad/truncate to this length

# =======================================================
# 3) Bandpass filter
# =======================================================
def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low, high = lowcut / nyq, highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs):
    b, a = butter_bandpass(lowcut, highcut, fs)
    return lfilter(b, a, data)

# =======================================================
# 4) EEG Dataset (shared LabelEncoder)
# =======================================================
class EEGDataset(Dataset):
    def __init__(self, df, base_path, le):
        self.df = df.reset_index(drop=True)
        self.base_path = base_path
        self.le = le
        if 'label' in df:
            self.labels = self.le.transform(df['label'])
        else:
            self.labels = None

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        dataset = 'train' if row['id'] <= 4800 else 'validation' if row['id'] <= 4900 else 'test'
        eeg_path = f"{self.base_path}/{row['task']}/{dataset}/{row['subject_id']}/{row['trial_session']}/EEGdata.csv"
        eeg_data = pd.read_csv(eeg_path)

        samples_per_trial = 2250 if row['task'] == 'MI' else 1750
        start = (row['trial'] - 1) * samples_per_trial
        end = start + samples_per_trial

        # Extract channels [C, T]
        trial = eeg_data.iloc[start:end][CHANNELS].values.T  # (4, T)

        # Pad or truncate
        if trial.shape[1] < MAX_LEN:
            pad_width = MAX_LEN - trial.shape[1]
            trial = np.pad(trial, ((0, 0), (0, pad_width)), mode='constant')
        elif trial.shape[1] > MAX_LEN:
            trial = trial[:, :MAX_LEN]

        # Bandpass
        mu = bandpass_filter(trial, MU_BAND[0], MU_BAND[1], FS)
        beta = bandpass_filter(trial, BETA_BAND[0], BETA_BAND[1], FS)

        # Improved features: mean, variance, log power
        means = np.mean(trial, axis=1)
        variances = np.var(trial, axis=1)
        diff_c3_c4 = means[1] - means[3]
        mu_power = np.mean(mu ** 2, axis=1)
        beta_power = np.mean(beta ** 2, axis=1)
        log_mu_power = np.log1p(mu_power)
        log_beta_power = np.log1p(beta_power)
        tabular = np.concatenate([means, variances, [diff_c3_c4], mu_power, beta_power, log_mu_power, log_beta_power]).astype(np.float32)

        waveform = torch.tensor(trial, dtype=torch.float32)
        tabular = torch.tensor(tabular, dtype=torch.float32)

        if self.labels is not None:
            label = torch.tensor(self.labels[idx], dtype=torch.long)
            return waveform, tabular, label
        else:
            return waveform, tabular

# =======================================================
# 5) Improved CNN + MLP Model
# =======================================================
class CNN_MLP(nn.Module):
    def __init__(self, in_channels, tabular_dim, num_classes):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels, 32, 7, padding=3),
            nn.ReLU(),
            nn.Conv1d(32, 64, 7, padding=3),
            nn.ReLU(),
            nn.Conv1d(64, 128, 7, padding=3),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten()
        )
        self.mlp = nn.Sequential(
            nn.Linear(tabular_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.final = nn.Linear(128 + 32, num_classes)

    def forward(self, x_wave, x_tab):
        x1 = self.cnn(x_wave)
        x2 = self.mlp(x_tab)
        x = torch.cat([x1, x2], dim=1)
        return self.final(x)

# =======================================================
# 6) Prepare MI data only
# =======================================================
train_df = pd.read_csv(f"{BASE_PATH}/train.csv")
val_df = pd.read_csv(f"{BASE_PATH}/validation.csv")

# Filter MI only
train_df = train_df[train_df['task'] == 'MI']
val_df = val_df[val_df['task'] == 'MI']

print("Train MI shape:", train_df.shape)
print("Val MI shape:", val_df.shape)

# Shared LabelEncoder
all_labels = pd.concat([train_df['label'], val_df['label']])
le = LabelEncoder()
le.fit(all_labels)

print("Classes:", le.classes_)

train_set = EEGDataset(train_df, BASE_PATH, le)
val_set = EEGDataset(val_df, BASE_PATH, le)

train_loader = DataLoader(train_set, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False, num_workers=2)

# =======================================================
# 7) Training Setup
# =======================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN_MLP(
    in_channels=len(CHANNELS),
    tabular_dim=4 + 4 + 1 + 4 + 4 + 4 + 4,  # means, vars, diff, mu, beta, log_mu, log_beta
    num_classes=len(le.classes_)
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

best_val_f1 = 0
patience = 15
patience_counter = 0

history = {'train_loss': [], 'val_loss': [], 'val_f1': []}

# =======================================================
# 8) Train with Early Stopping
# =======================================================
for epoch in range(100):
    model.train()
    train_loss = 0.0
    for x_wave, x_tab, y in train_loader:
        x_wave, x_tab, y = x_wave.to(device), x_tab.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x_wave, x_tab)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * y.size(0)

    train_loss /= len(train_loader.dataset)

    model.eval()
    val_loss = 0.0
    y_true, y_pred = [], []
    with torch.no_grad():
        for x_wave, x_tab, y in val_loader:
            x_wave, x_tab, y = x_wave.to(device), x_tab.to(device), y.to(device)
            out = model(x_wave, x_tab)
            loss = criterion(out, y)
            val_loss += loss.item() * y.size(0)
            y_true.extend(y.cpu().numpy())
            y_pred.extend(out.argmax(dim=1).cpu().numpy())

    val_loss /= len(val_loader.dataset)
    val_f1 = f1_score(y_true, y_pred, average='macro')

    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_f1'].append(val_f1)

    scheduler.step(val_loss)

    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f} | Val Loss={val_loss:.4f} | Val F1={val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("✅ Early stopping triggered!")
            break

# =======================================================
# 9) Save logs
# =======================================================
np.savez('training_log.npz', **history)
print("✅ Training log saved: training_log.npz")

Train MI shape: (2400, 6)
Val MI shape: (50, 6)
Classes: ['Left' 'Right']
Epoch 1: Train Loss=296339.5184 | Val Loss=154853.1388 | Val F1=0.6124
Epoch 2: Train Loss=88300.6461 | Val Loss=75875.0313 | Val F1=0.3056
Epoch 3: Train Loss=66881.3079 | Val Loss=46775.6937 | Val F1=0.4624
Epoch 4: Train Loss=41468.1601 | Val Loss=34253.2384 | Val F1=0.3633
Epoch 5: Train Loss=33030.3793 | Val Loss=47311.9820 | Val F1=0.3056
Epoch 6: Train Loss=27537.2437 | Val Loss=42371.2440 | Val F1=0.3779
Epoch 7: Train Loss=20874.0005 | Val Loss=32207.4491 | Val F1=0.4900
Epoch 8: Train Loss=17902.8322 | Val Loss=27178.2395 | Val F1=0.4419
Epoch 9: Train Loss=15305.1729 | Val Loss=41492.5540 | Val F1=0.3056
Epoch 10: Train Loss=13670.9867 | Val Loss=22681.2041 | Val F1=0.4419
Epoch 11: Train Loss=11576.2297 | Val Loss=30003.8821 | Val F1=0.5192
Epoch 12: Train Loss=11960.9229 | Val Loss=11019.2772 | Val F1=0.3905
Epoch 13: Train Loss=10550.5656 | Val Loss=19623.2946 | Val F1=0.3990
Epoch 14: Train Loss=11

In [6]:
rm -r /kaggle/working/*

rm: cannot remove '/kaggle/working/*': No such file or directory


In [3]:
print(train_df['label'].value_counts())
print(val_df['label'].value_counts())

label
Right    1213
Left     1187
Name: count, dtype: int64
label
Left     28
Right    22
Name: count, dtype: int64


In [11]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from tqdm import tqdm

# -----------------------------------------------------------
# 1) Dataset
# -----------------------------------------------------------

class WaveTabularDataset(Dataset):
    def __init__(self, wave_data, tabular_data, labels):
        self.wave_data = wave_data  # (N, L)
        self.tabular_data = tabular_data  # (N, F)
        self.labels = labels  # (N,)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        wave = torch.tensor(self.wave_data[idx], dtype=torch.float32).unsqueeze(0)  # (1, L)
        tab = torch.tensor(self.tabular_data[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return wave, tab, label

# -----------------------------------------------------------
# 2) Hybrid Model
# -----------------------------------------------------------

class HybridNet(nn.Module):
    def __init__(self, tab_input_dim=6, tab_hidden_dim=32, wave_output_dim=64, num_classes=2):
        super(HybridNet, self).__init__()

        # Wave CNN branch
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(16, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Flatten(),
            nn.Linear(32 * 125, wave_output_dim),  # adjust to match your signal length!
            nn.ReLU()
        )

        # Tabular MLP branch
        self.mlp = nn.Sequential(
            nn.Linear(tab_input_dim, tab_hidden_dim),
            nn.BatchNorm1d(tab_hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(tab_hidden_dim, tab_hidden_dim),
            nn.BatchNorm1d(tab_hidden_dim),
            nn.ReLU()
        )

        # Fusion MLP
        self.final = nn.Sequential(
            nn.Linear(wave_output_dim + tab_hidden_dim, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, x_wave, x_tab):
        x1 = self.cnn(x_wave)
        x2 = self.mlp(x_tab)
        x = torch.cat([x1, x2], dim=1)
        return self.final(x)

# -----------------------------------------------------------
# 3) Prepare Data (Example)
# -----------------------------------------------------------

# Simulate your wave and tabular MI data here:
# Replace with your real data
N_train, N_val = 2400, 50
wave_length = 500  # adjust to your real wave length
tab_features = 6

# Fake data for illustration
X_wave_train = np.random.randn(N_train, wave_length)
X_tab_train = np.random.randn(N_train, tab_features)
y_train = np.random.randint(0, 2, N_train)

X_wave_val = np.random.randn(N_val, wave_length)
X_tab_val = np.random.randn(N_val, tab_features)
y_val = np.random.randint(0, 2, N_val)

# Scale tabular MI data
scaler = StandardScaler()
X_tab_train = scaler.fit_transform(X_tab_train)
X_tab_val = scaler.transform(X_tab_val)

print(f"Train MI shape: {X_tab_train.shape}")
print(f"Val MI shape: {X_tab_val.shape}")

# -----------------------------------------------------------
# 4) Loaders
# -----------------------------------------------------------

train_ds = WaveTabularDataset(X_wave_train, X_tab_train, y_train)
val_ds = WaveTabularDataset(X_wave_val, X_tab_val, y_val)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)

# -----------------------------------------------------------
# 5) Training Loop
# -----------------------------------------------------------

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = HybridNet(tab_input_dim=tab_features).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

best_f1 = 0
patience = 10
counter = 0

train_log = {
    "train_loss": [],
    "val_loss": [],
    "val_f1": []
}

for epoch in range(1, 101):
    model.train()
    train_losses = []
    for wave, tab, label in train_loader:
        wave, tab, label = wave.to(device), tab.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(wave, tab)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    # Validation
    model.eval()
    val_losses = []
    all_preds, all_labels = [], []
    with torch.no_grad():
        for wave, tab, label in val_loader:
            wave, tab, label = wave.to(device), tab.to(device), label.to(device)
            output = model(wave, tab)
            loss = criterion(output, label)
            val_losses.append(loss.item())
            preds = output.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(label.cpu().numpy())

    avg_train_loss = np.mean(train_losses)
    avg_val_loss = np.mean(val_losses)
    val_f1 = f1_score(all_labels, all_preds, average="macro")

    train_log["train_loss"].append(avg_train_loss)
    train_log["val_loss"].append(avg_val_loss)
    train_log["val_f1"].append(val_f1)

    print(f"Epoch {epoch}: Train Loss={avg_train_loss:.4f} | Val Loss={avg_val_loss:.4f} | Val F1={val_f1:.4f}")

    # Early stopping
    if val_f1 > best_f1:
        best_f1 = val_f1
        counter = 0
        torch.save(model.state_dict(), "best_model.pt")
    else:
        counter += 1
        if counter >= patience:
            print("✅ Early stopping triggered!")
            break

# Save log
np.savez("training_log.npz", **train_log)
print("✅ Training log saved: training_log.npz")


Train MI shape: (2400, 6)
Val MI shape: (50, 6)
Epoch 1: Train Loss=0.7331 | Val Loss=0.7601 | Val F1=0.3350
Epoch 2: Train Loss=0.6687 | Val Loss=0.8044 | Val F1=0.5066
Epoch 3: Train Loss=0.5496 | Val Loss=1.2042 | Val F1=0.3316
Epoch 4: Train Loss=0.4290 | Val Loss=1.1002 | Val F1=0.5716
Epoch 5: Train Loss=0.3097 | Val Loss=1.1869 | Val F1=0.4982
Epoch 6: Train Loss=0.1894 | Val Loss=1.2206 | Val F1=0.4792
Epoch 7: Train Loss=0.1278 | Val Loss=1.5125 | Val F1=0.5166
Epoch 8: Train Loss=0.0905 | Val Loss=1.9723 | Val F1=0.5383
Epoch 9: Train Loss=0.0627 | Val Loss=2.1319 | Val F1=0.5166
Epoch 10: Train Loss=0.0466 | Val Loss=2.2201 | Val F1=0.5593
Epoch 11: Train Loss=0.0383 | Val Loss=2.0463 | Val F1=0.5000
Epoch 12: Train Loss=0.0452 | Val Loss=1.9553 | Val F1=0.5785
Epoch 13: Train Loss=0.0536 | Val Loss=3.1011 | Val F1=0.5572
Epoch 14: Train Loss=0.0447 | Val Loss=2.4327 | Val F1=0.5398
Epoch 15: Train Loss=0.0318 | Val Loss=2.4301 | Val F1=0.5074
Epoch 16: Train Loss=0.0401 | V

In [19]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm

# ------------------------
# 1️⃣  Fake data for demo 
# (replace with your real data)
# ------------------------
np.random.seed(0)
N = 2500
T = 128  # time steps
C = 3    # channels

X_wave = np.random.randn(N, C, T)
X_tab = np.random.randn(N, 6)  # your MI features
y = np.random.randint(0, 2, size=N)

# ------------------------
# 2️⃣  Extract TS stats 
# ------------------------
def extract_ts_features(X):
    """
    X: [N, C, T]
    Return: [N, C * num_stats]
    """
    mean = X.mean(axis=2)
    std = X.std(axis=2)
    min_ = X.min(axis=2)
    max_ = X.max(axis=2)
    median = np.median(X, axis=2)
    ptp = X.ptp(axis=2)  # peak-to-peak
    # concat along last axis
    features = np.concatenate([mean, std, min_, max_, median, ptp], axis=1)
    return features

X_wave_feat = extract_ts_features(X_wave)
print("Time series stats shape:", X_wave_feat.shape)

# Combine with MI features
X_combined = np.concatenate([X_tab, X_wave_feat], axis=1)
print("Final tabular shape:", X_combined.shape)

# ------------------------
# 3️⃣  Train/val split
# ------------------------
X_train, X_val, y_train, y_val = train_test_split(
    X_combined, y, test_size=0.05, random_state=42, stratify=y
)

print(f"Train shape: {X_train.shape}")
print(f"Val shape: {X_val.shape}")

# ------------------------
# 4️⃣  PyTorch Dataset & Loader
# ------------------------
class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds = TabularDataset(X_train, y_train)
val_ds = TabularDataset(X_val, y_val)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)

# ------------------------
# 5️⃣  Simple MLP Model
# ------------------------
class MLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(32, num_classes)
        )
    def forward(self, x):
        return self.net(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP(input_dim=X_combined.shape[1], num_classes=2).to(device)

# ------------------------
# 6️⃣  Loss & Optimizer
# ------------------------
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

# ------------------------
# 7️⃣  Training loop with early stopping
# ------------------------
best_f1 = 0
patience = 100
counter = 0
EPOCHS = 100

for epoch in range(1, EPOCHS+1):
    # Train
    model.train()
    train_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)
    train_loss /= len(train_loader.dataset)

    # Val
    model.eval()
    val_loss = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            loss = criterion(out, yb)
            val_loss += loss.item() * xb.size(0)
            preds = out.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(yb.cpu().numpy())
    val_loss /= len(val_loader.dataset)
    val_f1 = f1_score(all_labels, all_preds)

    print(f"Epoch {epoch}: Train Loss={train_loss:.4f} | Val Loss={val_loss:.4f} | Val F1={val_f1:.4f}")

    if val_f1 > best_f1:
        best_f1 = val_f1
        counter = 0
        torch.save(model.state_dict(), "best_mlp.pth")
    else:
        counter += 1
        if counter >= patience:
            print("✅ Early stopping triggered!")
            break

print(f"Best Val F1: {best_f1:.4f}")



Time series stats shape: (2500, 18)
Final tabular shape: (2500, 24)
Train shape: (2375, 24)
Val shape: (125, 24)
Epoch 1: Train Loss=0.7443 | Val Loss=0.6992 | Val F1=0.4211
Epoch 2: Train Loss=0.7140 | Val Loss=0.6946 | Val F1=0.4038
Epoch 3: Train Loss=0.7057 | Val Loss=0.6970 | Val F1=0.4103
Epoch 4: Train Loss=0.7009 | Val Loss=0.6969 | Val F1=0.1579
Epoch 5: Train Loss=0.7030 | Val Loss=0.6988 | Val F1=0.3529
Epoch 6: Train Loss=0.6966 | Val Loss=0.6987 | Val F1=0.3011
Epoch 7: Train Loss=0.6952 | Val Loss=0.6972 | Val F1=0.3208
Epoch 8: Train Loss=0.6918 | Val Loss=0.6962 | Val F1=0.2529
Epoch 9: Train Loss=0.6948 | Val Loss=0.6961 | Val F1=0.2637
Epoch 10: Train Loss=0.6956 | Val Loss=0.6942 | Val F1=0.4444
Epoch 11: Train Loss=0.6947 | Val Loss=0.6936 | Val F1=0.3111
Epoch 12: Train Loss=0.6928 | Val Loss=0.6945 | Val F1=0.3107
Epoch 13: Train Loss=0.6948 | Val Loss=0.6943 | Val F1=0.1928
Epoch 14: Train Loss=0.6914 | Val Loss=0.6955 | Val F1=0.3469
Epoch 15: Train Loss=0.6930 

In [ ]:
# =======================================================
# ✅ 1) Setup and imports
# =======================================================

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay
from scipy.signal import butter, lfilter
from mne.decoding import CSP
import matplotlib.pyplot as plt

torch.manual_seed(42)
np.random.seed(42)

# =======================================================
# ✅ 2) Config
# =======================================================

BASE_PATH = '/kaggle/input/mtcaic3'
CHANNELS = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
FS = 250
MU_BAND = (8, 13)
BETA_BAND = (13, 30)
MAX_LEN = 2250
SHORT_LEN = 500
N_CSP_COMPONENTS = 2

# =======================================================
# ✅ 3) Bandpass filter
# =======================================================

def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low, high = lowcut / nyq, highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs):
    b, a = butter_bandpass(lowcut, highcut, fs)
    return lfilter(b, a, data)

# =======================================================
# ✅ 4) Load data & label encoder
# =======================================================

train_df = pd.read_csv(f"{BASE_PATH}/train.csv")
val_df = pd.read_csv(f"{BASE_PATH}/validation.csv")

train_df = train_df[train_df['task'] == 'MI']
val_df = val_df[val_df['task'] == 'MI']

print("Train MI shape:", train_df.shape)
print("Val MI shape:", val_df.shape)

le = LabelEncoder()
le.fit(pd.concat([train_df['label'], val_df['label']]))
print("Classes:", le.classes_)

# =======================================================
# ✅ 5) Fit CSP
# =======================================================

X_csp, y_csp = [], []
for _, row in train_df.iterrows():
    eeg_path = f"{BASE_PATH}/{row['task']}/train/{row['subject_id']}/{row['trial_session']}/EEGdata.csv"
    eeg_data = pd.read_csv(eeg_path)
    start = (row['trial'] - 1) * 2250
    end = start + 2250
    trial = eeg_data.iloc[start:end][CHANNELS].values.T
    X_csp.append(trial[:, :SHORT_LEN])
    y_csp.append(le.transform([row['label']])[0])

X_csp = np.stack(X_csp)
y_csp = np.array(y_csp)
print("X_csp shape:", X_csp.shape)
print("y_csp shape:", y_csp.shape)

csp = CSP(n_components=N_CSP_COMPONENTS, reg='ledoit_wolf', log=True)
csp.fit(X_csp, y_csp)
print("✅ CSP fitted.")

# =======================================================
# ✅ 6) Dataset
# =======================================================

class EEGDataset(Dataset):
    def __init__(self, df, base_path, le, csp):
        self.df = df.reset_index(drop=True)
        self.base_path = base_path
        self.le = le
        self.csp = csp
        if 'label' in df:
            self.labels = self.le.transform(df['label'])
        else:
            self.labels = None

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        dataset = 'train' if row['id'] <= 4800 else 'validation'
        eeg_path = f"{self.base_path}/{row['task']}/{dataset}/{row['subject_id']}/{row['trial_session']}/EEGdata.csv"
        eeg_data = pd.read_csv(eeg_path)
        start = (row['trial'] - 1) * 2250
        end = start + 2250
        trial = eeg_data.iloc[start:end][CHANNELS].values.T

        if trial.shape[1] < MAX_LEN:
            pad = MAX_LEN - trial.shape[1]
            trial = np.pad(trial, ((0, 0), (0, pad)), mode='constant')
        elif trial.shape[1] > MAX_LEN:
            trial = trial[:, :MAX_LEN]

        mu = bandpass_filter(trial, MU_BAND[0], MU_BAND[1], FS)
        beta = bandpass_filter(trial, BETA_BAND[0], BETA_BAND[1], FS)

        means = np.mean(trial, axis=1)
        vars_ = np.var(trial, axis=1)
        diff = means[CHANNELS.index('C3')] - means[CHANNELS.index('C4')]
        mu_power = np.mean(mu**2, axis=1)
        beta_power = np.mean(beta**2, axis=1)
        log_mu = np.log1p(mu_power)
        log_beta = np.log1p(beta_power)

        tabular = np.concatenate([means, vars_, [diff], mu_power, beta_power, log_mu, log_beta]).astype(np.float32)
        csp_feats = self.csp.transform(trial[:, :SHORT_LEN][np.newaxis, ...]).squeeze().astype(np.float32)

        waveform = torch.tensor(trial, dtype=torch.float32)
        tabular = torch.tensor(tabular, dtype=torch.float32)
        csp_feats = torch.tensor(csp_feats, dtype=torch.float32)

        if self.labels is not None:
            label = torch.tensor(self.labels[idx], dtype=torch.long)
            return waveform, tabular, csp_feats, label
        else:
            return waveform, tabular, csp_feats

# =======================================================
# ✅ 7) Dataloaders
# =======================================================

train_set = EEGDataset(train_df, BASE_PATH, le, csp)
val_set = EEGDataset(val_df, BASE_PATH, le, csp)

train_loader = DataLoader(train_set, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False, num_workers=2)

# =======================================================
# ✅ 8) Model: CNN + MLP + CSP
# =======================================================

class CNN_MLP_CSP(nn.Module):
    def __init__(self, in_channels, tabular_dim, csp_dim, num_classes):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels, 32, 7, padding=3),
            nn.ReLU(),
            nn.Conv1d(32, 64, 7, padding=3),
            nn.ReLU(),
            nn.Conv1d(64, 128, 7, padding=3),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten()
        )
        self.mlp = nn.Sequential(
            nn.Linear(tabular_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.csp_layer = nn.Sequential(
            nn.Linear(csp_dim, 16),
            nn.ReLU()
        )
        self.final = nn.Linear(128 + 32 + 16, num_classes)

    def forward(self, x_wave, x_tab, x_csp):
        x1 = self.cnn(x_wave)
        x2 = self.mlp(x_tab)
        x3 = self.csp_layer(x_csp)
        x = torch.cat([x1, x2, x3], dim=1)
        return self.final(x)

# ✅ Corrected tabular_dim
TABULAR_DIM = 8 * 6 + 1  # 49

# =======================================================
# ✅ 9) Train setup
# =======================================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN_MLP_CSP(
    in_channels=len(CHANNELS),
    tabular_dim=TABULAR_DIM,
    csp_dim=N_CSP_COMPONENTS,
    num_classes=len(le.classes_)
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

best_f1 = 0
patience = 15
counter = 0

history = {'train_loss': [], 'val_loss': [], 'val_f1': []}

# =======================================================
# ✅ 10) Training loop
# =======================================================

for epoch in range(100):
    model.train()
    train_loss = 0.0
    for x_wave, x_tab, x_csp, y in train_loader:
        x_wave, x_tab, x_csp, y = x_wave.to(device), x_tab.to(device), x_csp.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x_wave, x_tab, x_csp)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * y.size(0)
    train_loss /= len(train_loader.dataset)

    model.eval()
    val_loss = 0.0
    y_true, y_pred = [], []
    with torch.no_grad():
        for x_wave, x_tab, x_csp, y in val_loader:
            x_wave, x_tab, x_csp, y = x_wave.to(device), x_tab.to(device), x_csp.to(device), y.to(device)
            out = model(x_wave, x_tab, x_csp)
            val_loss += criterion(out, y).item() * y.size(0)
            y_true.extend(y.cpu().numpy())
            y_pred.extend(out.argmax(dim=1).cpu().numpy())

    val_loss /= len(val_loader.dataset)
    val_f1 = f1_score(y_true, y_pred, average='macro')
    scheduler.step(val_loss)

    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_f1'].append(val_f1)

    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f} | Val Loss={val_loss:.4f} | Val F1={val_f1:.4f}")

    if val_f1 > best_f1:
        best_f1 = val_f1
        counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        counter += 1
        if counter >= patience:
            print("✅ Early stopping triggered.")
            break

np.savez('training_log.npz', **history)
print("✅ Training log saved.")

# =======================================================
# ✅ 11) Confusion Matrix
# =======================================================

model.load_state_dict(torch.load('best_model.pth'))
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for x_wave, x_tab, x_csp, y in val_loader:
        x_wave, x_tab, x_csp, y = x_wave.to(device), x_tab.to(device), x_csp.to(device), y.to(device)
        out = model(x_wave, x_tab, x_csp)
        y_true.extend(y.cpu().numpy())
        y_pred.extend(out.argmax(dim=1).cpu().numpy())

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
fig, ax = plt.subplots(figsize=(6,6))
disp.plot(ax=ax, cmap='Blues')
plt.title('Confusion Matrix')
plt.show()

Train MI shape: (2400, 6)
Val MI shape: (50, 6)
Classes: ['Left' 'Right']
X_csp shape: (2400, 8, 500)
y_csp shape: (2400,)
Computing rank from data with rank=None
    Using tolerance 1.7e+06 (2.2e-16 eps * 8 dim * 9.7e+20  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=0 covariance using LEDOIT_WOLF
Done.
Estimating class=1 covariance using LEDOIT_WOLF
Done.
✅ CSP fitted.
Epoch 1: Train Loss=200686.4249 | Val Loss=152534.4675 | Val F1=0.4156
Epoch 2: Train Loss=82880.4779 | Val Loss=61687.7453 | Val F1=0.4391
Epoch 3: Train Loss=39837.5791 | Val Loss=45520.0488 | Val F1=0.4907
Epoch 4: Train Loss=35179.7054 | Val Loss=15817.2716 | Val F1=0.3961
Epoch 5: Train Loss=23479.1575 | Val Loss=30597.1006 | Val F1=0.3990
Epoch 6: Train Loss=16628.1290 | Val Loss=8919.7069 | Val F1=0.4949
Epoch 7: Train Loss=15095.2325 | Val Loss=39823.5575 | Val F1=0.3990
Epoch 8: Train Loss=11274.